In [5]:
#Lecture B
#last year final exam
#final practice
#Q1 (unit, may need to divide 1000)
def cash_flow_present_value(cash_flow, effective_rate):
    #"""Enter effective rate in percentage"""
    if isinstance(effective_rate, list) or isinstance(effective_rate, tuple):
        spr = [1] + [1 + r/100 for r in effective_rate]
        p = 0.
        for i, ai in enumerate(cash_flow):
            p += ai / (spr[i]) ** i
        return p
    r = 1 + effective_rate / 100
    res = 0
    for i, ai in enumerate(cash_flow):
        res += ai / r ** i
    return res

class Bond:
    #"""Implementation of a bond. A face value $30000, 9% bond that matures in 30 periods (years) that pays its coupons twice a period (year) should be entered as Bond(9, 30, 2, 300)"""
    def __init__(self, coupon_percentage, maturity_years, coupon_freq, face_value):
        self.percentage = coupon_percentage / 100
        self.maturity_years = maturity_years
        self.coupon_freq = coupon_freq
        self.face_value = face_value
        flow = [0] + [coupon_percentage / coupon_freq] * maturity_years * coupon_freq
        flow[-1] += face_value
        self.flow = tuple(flow)
    
    @property
    def coupon_payment(self):
        return self.percentage * self.face_value / self.coupon_freq

def QM_duration(cash_flow_or_bond, rates, compound_freq):
    #"""Enter a list of spot rates (in percentage) to calculate the quasi-modified duration.
    #Keep in mind if the price at rates = sk + y is P(y), then P'(0) = -Dq * P(0)

    #This trims the first term to keep consistent with the bond calculations. i.e. following the book. calculating the
    #quasimodified duration for (x0, x1, ..., xn) where x0 is the flow on year 0
    
    #You can actually feed a bond into this function and we will handle it for you"""
    
    if isinstance(cash_flow_or_bond, Bond):
        cash_flow = cash_flow_or_bond.flow
    else:
        cash_flow = cash_flow_or_bond
    
    assert len(cash_flow) <= len(rates) + 1, f"Wrong number of periods: we have {len(rates)} years of spot rates but {len(cash_flow)} years bond"

    # First calculate the pv of cash flow
    spr = [1] + [1 + r/100 for r in rates]
    res = 0
    for i, ai in enumerate(cash_flow):
        res += ai / (spr[i]) ** i
    PV = res

    x = cash_flow[1:]
    s = [1 + r/100 for r in rates]
    m = compound_freq
    n = len(cash_flow) - 1
    res = 0.
    for _k in range(n):
        k = _k + 1
        rs = (k/m) * x[_k] * (s[_k]) ** (-k-1)
        res += rs
    return res/PV

def immunize_parallel(bond1: Bond, bond2: Bond, obligation_cash_flow, rates, verbose: bool = False):
    # Calculate the PVob, Dob of obligation
    # Calculate the PV1, D1 of bond1
    # Calculate the PV2, D2 of bond2

    obligation = obligation_cash_flow

    # First do error checking
    num_years_total = max(bond1.maturity_years, bond2.maturity_years, len(obligation) - 1)
    enough_rates = len(rates) >= num_years_total
    if not enough_rates:
        raise ValueError(f"Expected at least {num_years_total} years of rates, found {len(rates)}")

    PVob = cash_flow_present_value(obligation, rates)
    PV1 = cash_flow_present_value(bond1.flow, rates)
    PV2 = cash_flow_present_value(bond2.flow, rates)
    
    Dob = QM_duration(obligation, rates, 1)
    D1 = QM_duration(bond1, rates, 1)
    D2 = QM_duration(bond2, rates, 1)

    if verbose:
        print(f"Duration of bond 1: {D1}")
        print(f"Duration of bond 2: {D2}")
        print(f"Duration of obligation: {Dob}")
        print(f"Present value of bond 1: {PV1}")
        print(f"Present value of bond 2: {PV2}")
        print(f"Present value of obligation: {PVob}")

    # Solve for
    # PV1   PV2     x1 = PVob
    # PV1D1 PV2D2   x2 = PVob Dob

    # Hence
    # x1 =  |PV1   PV2  |-1  |PVob    |
    # x2 =  |PV1D1 PV2D2|    |PVob Dob|
    det = PV1 * PV2 * D2 - PV2 * PV1 * D1
    if abs(det) < 1e-5:
        raise ValueError("This portfolio cannot be immunized - the equations have no solutions")

    x1 = 1/det * (PV2 * PVob * D2 - PVob * Dob * PV2)
    x2 = 1/det * (PV1 * PVob * Dob - PVob * D1 * PV1)

    if verbose:
        print(f"Buy {x1} units of bond 1 and {x2} units of bond 2")

    return x1, x2


## The inputs: in percentage
spot_rates = [5.25, 5.50, 5.75, 6.00, 6.25]
bond1 = Bond(6, 5, 1, 100)
bond2 = Bond(10, 4, 1, 100)
obligation = (0, 50000, 51000 + 51000, 52000 + 52000, 53000 + 53000, 54000)
immunize_parallel(bond1, bond2, obligation, spot_rates)


(-2216.560382053845, 5003.967355938829)

In [6]:
#Q2
def new_paymt(P, n1, m, r1, r2, n2): 
    A1 = r1/m*P*(1-1/(1+r1/m)**(n1*m))**(-1)
    print('Orginal Payment:', A1)
    P1 = A1/(r1/m)*(1-1/(1+r1/m)**((n1-n2)*m))
    A2 = r2/m*P1*(1-1/(1+r2/m)**((n1-n2)*m))**(-1)
    return A2

new_paymt(P=1000000, n1=30, m=12, r1=0.05, r2=0.08, n2=5)

Orginal Payment: 5368.216230121399


7087.490482768596

In [3]:
#Q3
# Parameters as per the user's description
initial_cost = 26000  
replacement_cost = 26000
annual_increase = 1000
interest_rate = 0.06
operating_cost_first_year = 2000

# Define the operating costs for a maximum of 30 years
years = 30
operating_costs = [operating_cost_first_year + i * annual_increase for i in range(years)]

# Function to calculate present value of costs
def present_value(cost, rate, year):
    return cost / ((1 + rate) ** year)

# Calculate the present value of the operating costs for each year
present_values_operating = [present_value(oc, interest_rate, year) for year, oc in enumerate(operating_costs, start=1)]

# Function to calculate the total cost for a given replacement period n
def total_cost(n):
    # Present value of initial cost
    pv_initial_cost = present_value(initial_cost, interest_rate, 0)
    # Sum of present values of operating costs up to year n
    pv_operating_costs = sum(present_values_operating[:n])
    # Total present value cost of owning the machine for n years (including replacement every n years indefinitely)
    # is the initial cost plus operating costs times the series sum of (1 - (1 + r)^(-n))^-1
    # This is the present value annuity factor for a perpetuity with payments every n years
    annuity_factor = (1 - (1 + interest_rate)**(-n))**(-1)
    total_pv_cost = (pv_initial_cost + pv_operating_costs) * annuity_factor
    return total_pv_cost

# Initialize the minimum cost to a large number and optimal n to None
min_cost = float('inf')
optimal_n = None

# Check for the optimal replacement period from 1 to 30 years
for n in range(1, years + 1):
    cost_n = total_cost(n)
    if cost_n < min_cost:
        min_cost = cost_n
        optimal_n = n

optimal_n, min_cost


(8, 156369.0359294578)

In [7]:
#Q4
def allocate_a(n, rb, rm, rf):   # $n dollars, a in market portfolio, n-a in risk-free asset, return a
    # n*rb = a*rm + (n-a)*rf
    return n*(rb-rf) / (rm-rf)

allocate_a(n=1000, rb=0.2, rm=0.16, rf=0.06)

1400.0

In [1]:
#Q5 同Q10 final exam practice
import numpy as np

# Covariance matrix V
V = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])

# Augmenting the covariance matrix with ones to account for the constraint
augmented_matrix = np.vstack([V, np.ones(V.shape[1])])
augmented_matrix = np.hstack([augmented_matrix, np.ones((augmented_matrix.shape[0], 1))])
augmented_matrix[-1, -1] = 0

# Right-hand side vector
b = np.zeros(V.shape[1] + 1)
b[-1] = 1

# Solving the linear system
solution = np.linalg.solve(augmented_matrix, b)

# Extracting the weights
weights = solution[:-1]

weights, np.dot(weights.T, np.dot(V, weights))

(array([ 0.0971984 ,  0.53230417, -0.15208691,  0.60777587, -0.08519154]),
 0.12381360777587194)

In [3]:
import numpy as np
V = np.array([
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])
r_bar = np.array([0.69, 0.65, 0.44, 0.36, 0.41]).reshape(-1, 1)
var_v = 0.04
beta = np.array([1.16, 0.70, 1.15, 0.98, 1.01]).reshape(-1, 1)
cov_v = beta * var_v

# Constructing the matrices for the linear system
A = np.block([
    [2 * V, -np.ones((5, 1)), -2*r_bar],
    [np.ones((1, 5)), 0, 0],
    [r_bar.T, 0, 0]
])
b = np.vstack((2 * cov_v, 1, 0.2))

# Solving the linear system
OPW_13 = np.linalg.solve(A, b)
OPW_13

array([[-0.57952824],
       [-0.07124196],
       [ 0.50596584],
       [ 0.91625997],
       [ 0.22854439],
       [ 0.35407861],
       [-0.16231041]])

In [2]:
#Q6 not right
import cvxpy as cp
import numpy as np

# Given data
V = np.array([  # covariance matrix
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])
r_bar = np.array([0.69, 0.65, 0.44, 0.36, 0.41])  # expected returns
target_return = 0.20  # Target portfolio mean return
variance_SnP500 = 0.04  # Variance of the S&P 500 index returns

# The variable we are solving for (portfolio weights)
weights = cp.Variable(5)

# The objective is to minimize the portfolio variance
risk = cp.quad_form(weights, V)
return_portfolio = r_bar @ weights
objective = cp.Minimize(risk)

# Constraints
constraints = [
    return_portfolio == 2 * target_return,  # Double the target return
    cp.sum(weights) == 1,  # Sum of weights equals 1
    weights >= -1  # Allow short positions (negative weights)
]

# Solving the problem
prob = cp.Problem(objective, constraints)
prob.solve()

# The optimal weights
optimal_weights = weights.value

# Output the optimal weights
print("The optimal weights for the portfolio are:")
print(optimal_weights)


The optimal weights for the portfolio are:
[-0.21472997  0.35810435  0.06266244  0.7540105   0.03995268]


In [8]:
#Q7
import numpy as np
from scipy.optimize import fsolve

# Function to calculate the price of a bond given its yield to maturity (YTM)
def bond_price(face_value, coupon_rate, n_periods, YTM):
    coupon = face_value * coupon_rate / 2  # Semi-annual coupon payment
    discount_factors = [(1 + YTM / 2)**-i for i in range(1, n_periods + 1)]
    price = sum(coupon * df for df in discount_factors) + face_value * discount_factors[-1]
    return price

# Function to calculate the yield to maturity of a bond
def bond_yield_to_maturity(price, face_value, coupon_rate, n_periods):
    # Function to be solved for the yield (the difference between market price and calculated price)
    func = lambda YTM: bond_price(face_value, coupon_rate, n_periods, YTM) - price
    # Initial guess for the yield (close to the coupon rate)
    YTM_initial_guess = coupon_rate
    # Solve for the yield to maturity
    YTM, = fsolve(func, YTM_initial_guess)
    return YTM

# Given values for the 5-year bond
price_5_year = 108  # Given market price of the 5-year bond
face_value = 100
coupon_rate_5_year = 0.04
n_periods_5_year = 5 * 2  # 5 years, semi-annual coupons

# Calculate the YTM for the 5-year bond
YTM_5_year = bond_yield_to_maturity(price_5_year, face_value, coupon_rate_5_year, n_periods_5_year)

# Now, calculate the price of the 10-year bond using the YTM of the 5-year bond
coupon_rate_10_year = 0.10
n_periods_10_year = 10 * 2  # 10 years, semi-annual coupons

# Calculate the price of the 10-year bond using the YTM of the 5-year bond
price_10_year = bond_price(face_value, coupon_rate_10_year, n_periods_10_year, YTM_5_year)

# Round the price to two decimal places
price_10_year_rounded = round(price_10_year, 2)

price_10_year_rounded

168.47

In [1]:
import numpy as np

def IRRPoly(cash_flow, m):
    coefficients = np.flip(cash_flow)
    roots = np.roots(coefficients)
    irr_poly = np.real(roots[(roots >= 0)  & (np.iscomplex(roots) == False)])[0]
    irr = m*(1 - irr_poly) / irr_poly
    return irr

# Bond characteristics
face_value = 100
coupon_payment = 4  #yearly
payments_per_year = 2
years_to_maturity = 5
current_price = 108

# Cash flows for the bond

cash_flows = (coupon_payment / payments_per_year)*np.ones(years_to_maturity * payments_per_year)
cash_flows[-1] += face_value
print(cash_flows)
cash_flows = np.insert(cash_flows, 0, -current_price)
print(cash_flows)

# Find YTM using IRRPoly function
ytm = IRRPoly( cash_flows, payments_per_year)

# Convert to a percentage and display the result
ytm_percentage = ytm * 100
print(f"The bond's yield to maturity is {ytm_percentage:.2f}%")

def bond_price(F, C, lamb, m, n):
    '''
    F is the face value of the bond
    C is the yearly coupon payment
    lamb is the yield of the bond 
    m is the number of coupon payments per year
    n is the number of coupon payments remaning
    '''
    r = lamb / m  # Semi-annual yield
    discount_factor = (1 + r) ** n
    bond_price = (C / r) * (1 - 1 / discount_factor) + F / discount_factor
    return bond_price

face_value = 100
coupon_rate = 0.1
periods=10
frequency = 2
coupon_payment_per_period = face_value * coupon_rate / frequency
bond_price(face_value, coupon_payment_per_period, ytm, frequency, periods * frequency)

[  2.   2.   2.   2.   2.   2.   2.   2.   2. 102.]
[-108.    2.    2.    2.    2.    2.    2.    2.    2.    2.  102.]
The bond's yield to maturity is 2.30%


168.4718119347604

In [4]:
#Q8
def sum_sharpe_ratios(num_assets):
    return sum(range(1, num_assets + 1))

def optimal_portfolio_weight(num_assets):
    return 1 / sum_sharpe_ratios(num_assets)

num_assets = 100

optimal_weight = optimal_portfolio_weight(num_assets)
optimal_weight_percentage = optimal_weight * 100
optimal_max_sharpe = optimal_weight_percentage * 100

print("The optimal portfolio weight for the risky asset with the maximum Sharpe ratio is approximately:",optimal_max_sharpe) 

#答案除以100

The optimal portfolio weight for the risky asset with the maximum Sharpe ratio is approximately: 1.9801980198019802


In [9]:
#Q9
from scipy.optimize import minimize

bonds = [
    Bond(0, 5, 1, 100),
    Bond(0, 10, 1, 100),
    Bond(0, 15, 1, 100),
    Bond(0, 20, 1, 100),
    Bond(0, 25, 1, 100),
    Bond(0, 30, 1, 100)
]

yield_curves = [1, 2.5, 2.5, 2.9, 3, 3.1]

def bond_price(bond, yield_to_maturity: float, remaining_periods = None):
    #"""Calculate the price of the bond based on its yield to maturity. The price and yield operations should be inverses to each other, given the same number of remaining periods until maturity.
    #Please enter your yield to maturity in percentages"""
    if remaining_periods is None:
        remaining_periods = bond.coupon_freq * bond.maturity_years
    
    m = bond.coupon_freq
    F = bond.face_value
    C = bond.percentage * bond.face_value
    y = yield_to_maturity / 100
    n = remaining_periods

    fac = 1 / (1 + y / m) ** n
    return F * fac + C / y * (1 - fac)

def treasury_yield_my_ass(bonds, yield_curves):
    prices = [bond_price(bond, rate) for bond, rate in zip(bonds, yield_curves)]

    # Lets say we have these weights
    s = [1, 0, 0, 0, 0, 0]

    def optimize_this(s):
        cost = sum(a * b for a, b in zip(s, prices))

        # In five years let's say we sell eveything
        y5 = 100 * s[0]
        for i in range(5):
            y5 += s[i+1] * prices[i]
        # We want biggest profit/cost ratio, hence smallest cost/profit
        return cost/y5

    ms = minimize(optimize_this, [1, 0, 0, 0, 0, 0], bounds = [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)])

    weights = ms.x
    return 1/optimize_this(weights)

five_year_max_return_ratio = treasury_yield_my_ass(bonds, yield_curves)

# Do the five year thing twice
1e5 * five_year_max_return_ratio * five_year_max_return_ratio

149588.78293578638

In [3]:
#Q10 不理解，题目一样结果不一样
import cvxpy as cp

# Given data
V = np.array([ # covariance matrix
    [0.21, 0.12, 0.17, 0.13, 0.16],
    [0.12, 0.15, 0.12, 0.10, 0.12],
    [0.17, 0.12, 0.19, 0.14, 0.15],
    [0.13, 0.10, 0.14, 0.15, 0.14],
    [0.16, 0.12, 0.15, 0.14, 0.21]
])
r_bar = np.array([0.69, 0.65, 0.44, 0.36, 0.41]) # expected returns

# The variable we are solving for
weights = cp.Variable(5)

# The objective is to minimize the portfolio variance
# and maximize the expected return
risk = cp.quad_form(weights, V)
return_portfolio = r_bar @ weights
objective = cp.Minimize(risk - return_portfolio)

# The sum of the weights must be 1
constraints = [cp.sum(weights) == 1]

# Solving the problem
prob = cp.Problem(objective, constraints)
prob.solve()

# The optimal weights
optimal_weights = weights.value

# Output the optimal weights
print("The optimal weights for the portfolio are:")
print(optimal_weights)

# Specifically, to output the weight for Meta Platforms, Inc. (formerly Facebook, ticker META):
AMZN_weight = optimal_weights[1]  # Assuming the 5th stock in the list is Meta/Facebook
print(f"The optimal weight for AMZN Platforms, Inc. (AMZN) is: {AMZN_weight:.2f}")


The optimal weights for the portfolio are:
[ 2.39308176  1.81446541 -1.7327044  -0.46855346 -1.00628931]
The optimal weight for AMZN Platforms, Inc. (AMZN) is: 1.81


In [ ]:
#6，10(感觉是对的)